# Spark DF - Data Cleaning
Now that you understand the basics of Spark DataFrames, let's move on to a more relevant example: Cleaning data using the PySpark API. 

This tutorial is adapted from Stack Overflow and the PySpark documentation. Find out more here:
- https://stackoverflow.com/questions/37749759/fill-pyspark-dataframe-column-null-values-with-average-value-from-same-column
- http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html

Objective: Let's explore our options when it comes to cleaning a dataset. We'll be using the same dataset used in the Spark DF - Basics tutorial. Remember, the dataset is used to predict whether or not a client will subscribe to a term deposit (deposit column) if called by the banks call centre reps. 

In [1]:
# Section must be included at the beginning of each new notebook. Remember to change the app name. 
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missing').getOrCreate()

In [2]:
# Importing data which has a header and automatically configuring the schema.
df = spark.read.csv('Datasets/dataframe_dataset.csv', header=True, inferSchema=True)

# Let's visually inspect the data. You may notice some nulls in the marital and balance column. 
df.show()

# Let's see how many rows of data we originally have.
print("Total data points:", df.count())

+---+-----------+--------+---------+-------+-------+----+--------+-------+
|age|        job| marital|education|balance|housing|loan|duration|deposit|
+---+-----------+--------+---------+-------+-------+----+--------+-------+
| 59|     admin.|    null|secondary|   2343|    yes|  no|    1042|    yes|
| 56|     admin.| married|secondary|   null|     no|  no|    1467|    yes|
| 41| technician| married|secondary|   1270|    yes|  no|    1389|    yes|
| 55|   services|    null|secondary|   null|    yes|  no|     579|    yes|
| 54|     admin.| married| tertiary|    184|     no|  no|     673|    yes|
| 42| management|    null| tertiary|      0|    yes| yes|     562|    yes|
| 56| management| married| tertiary|    830|    yes| yes|    1201|    yes|
| 60|    retired|    null|secondary|    545|    yes|  no|    1030|    yes|
| 37| technician| married|secondary|      1|    yes|  no|     608|    yes|
| 28|   services|    null|secondary|   null|    yes|  no|    1297|    yes|
| 38|     admin.|  single

## Dropping Rows - Missing Data Point 
Spark can drop any row with missing data. Let's see how that's done!

In [3]:
# 'na' stands for Not Available. Using na, we can then use drop. 
# After using show, you'll find that the rows with the null values are gone. 
df.na.drop().show()

# Let's see how many rows of data we have now. 
print("Total data points:", df.count())

+---+-----------+--------+---------+-------+-------+----+--------+-------+
|age|        job| marital|education|balance|housing|loan|duration|deposit|
+---+-----------+--------+---------+-------+-------+----+--------+-------+
| 41| technician| married|secondary|   1270|    yes|  no|    1389|    yes|
| 54|     admin.| married| tertiary|    184|     no|  no|     673|    yes|
| 56| management| married| tertiary|    830|    yes| yes|    1201|    yes|
| 37| technician| married|secondary|      1|    yes|  no|     608|    yes|
| 38|     admin.|  single|secondary|    100|    yes|  no|     786|    yes|
| 30|blue-collar| married|secondary|    309|    yes|  no|    1574|    yes|
| 29| management| married| tertiary|    199|    yes| yes|    1689|    yes|
| 46|blue-collar|  single| tertiary|    460|    yes|  no|    1102|    yes|
| 31| technician|  single| tertiary|    703|    yes|  no|     943|    yes|
| 35| management|divorced| tertiary|   3837|    yes|  no|    1084|    yes|
| 49|   services| married

In [4]:
# Oops! Forgot to assign the result to a variable. Let's try that again. Now you can see that 7 rows have been removed.
dropped_df = df.na.drop()
dropped_df.show()
print("Total data points:", dropped_df.count())

+---+-----------+--------+---------+-------+-------+----+--------+-------+
|age|        job| marital|education|balance|housing|loan|duration|deposit|
+---+-----------+--------+---------+-------+-------+----+--------+-------+
| 41| technician| married|secondary|   1270|    yes|  no|    1389|    yes|
| 54|     admin.| married| tertiary|    184|     no|  no|     673|    yes|
| 56| management| married| tertiary|    830|    yes| yes|    1201|    yes|
| 37| technician| married|secondary|      1|    yes|  no|     608|    yes|
| 38|     admin.|  single|secondary|    100|    yes|  no|     786|    yes|
| 30|blue-collar| married|secondary|    309|    yes|  no|    1574|    yes|
| 29| management| married| tertiary|    199|    yes| yes|    1689|    yes|
| 46|blue-collar|  single| tertiary|    460|    yes|  no|    1102|    yes|
| 31| technician|  single| tertiary|    703|    yes|  no|     943|    yes|
| 35| management|divorced| tertiary|   3837|    yes|  no|    1084|    yes|
| 49|   services| married

## Dropping Rows - Missing Field
What about data missing in a particular field? 

Using subset, we're able to specify that if data in a particular feature is missing, then the entire row should be dropped. Let's see how that works. 

In [5]:
missing_field_df = df.na.drop(subset="marital")
missing_field_df.show()
print("Total data points:", missing_field_df.count())

+---+-----------+--------+---------+-------+-------+----+--------+-------+
|age|        job| marital|education|balance|housing|loan|duration|deposit|
+---+-----------+--------+---------+-------+-------+----+--------+-------+
| 56|     admin.| married|secondary|   null|     no|  no|    1467|    yes|
| 41| technician| married|secondary|   1270|    yes|  no|    1389|    yes|
| 54|     admin.| married| tertiary|    184|     no|  no|     673|    yes|
| 56| management| married| tertiary|    830|    yes| yes|    1201|    yes|
| 37| technician| married|secondary|      1|    yes|  no|     608|    yes|
| 38|     admin.|  single|secondary|    100|    yes|  no|     786|    yes|
| 30|blue-collar| married|secondary|    309|    yes|  no|    1574|    yes|
| 29| management| married| tertiary|    199|    yes| yes|    1689|    yes|
| 46|blue-collar|  single| tertiary|    460|    yes|  no|    1102|    yes|
| 31| technician|  single| tertiary|    703|    yes|  no|     943|    yes|
| 35| management|divorced

Interesting! Looks like five data points in the marital feature are missing. 

## Filling Null Values
Instead of just exploring the quality of the data, we can also fill a missing value with some text. Let's see how that's done. 

In [6]:
# It's good practice to specify the column to be filled. 
# In this case, as we know that marital has some missing values, let's fill the null values with the text "UNDISCLOSED".
filled_df = df.na.fill('UNDISCLOSED', subset=['marital'])
filled_df.show()

+---+-----------+-----------+---------+-------+-------+----+--------+-------+
|age|        job|    marital|education|balance|housing|loan|duration|deposit|
+---+-----------+-----------+---------+-------+-------+----+--------+-------+
| 59|     admin.|UNDISCLOSED|secondary|   2343|    yes|  no|    1042|    yes|
| 56|     admin.|    married|secondary|   null|     no|  no|    1467|    yes|
| 41| technician|    married|secondary|   1270|    yes|  no|    1389|    yes|
| 55|   services|UNDISCLOSED|secondary|   null|    yes|  no|     579|    yes|
| 54|     admin.|    married| tertiary|    184|     no|  no|     673|    yes|
| 42| management|UNDISCLOSED| tertiary|      0|    yes| yes|     562|    yes|
| 56| management|    married| tertiary|    830|    yes| yes|    1201|    yes|
| 60|    retired|UNDISCLOSED|secondary|    545|    yes|  no|    1030|    yes|
| 37| technician|    married|secondary|      1|    yes|  no|     608|    yes|
| 28|   services|UNDISCLOSED|secondary|   null|    yes|  no|    

In [7]:
# For integer type features, you may want to replace the null value the average of that feature.
# Let's see how that's done using balance as an example.
from pyspark.sql.functions import mean

# Let's find the average. But the problem is, how do we get that information out of the DataFrame?
mean_balance = df.select(mean('balance')).show()

# We can use collect to return the DataFrame as a Python list. But this still doesn't look correct. 
# How do we remove all the outer layers and simply extract the value?
mean_balance = df.select(mean('balance')).collect()
print(mean_balance)

# In this case, we'll need to use indexes. As it's a list, each piece of data has a unique place.
# If we look at the first place (0), you see that the outer brackets are stripped.
mean_balance = mean_balance[0]
print(mean_balance)

# If we refer to the index again, the rest of outer information is stripped and we're finally left with the raw value!
mean_balance = mean_balance[0]
print(mean_balance)

+------------------+
|      avg(balance)|
+------------------+
|1074.5520833333333|
+------------------+

[Row(avg(balance)=1074.5520833333333)]
Row(avg(balance)=1074.5520833333333)
1074.5520833333333


In [8]:
# Now, we can fill the missing values with the mean. 
# As the mean is 1074, you'll see 1074 instead of null in the balance column.
filled_df = filled_df.na.fill(mean_balance, subset=['balance'])
filled_df.show()

+---+-----------+-----------+---------+-------+-------+----+--------+-------+
|age|        job|    marital|education|balance|housing|loan|duration|deposit|
+---+-----------+-----------+---------+-------+-------+----+--------+-------+
| 59|     admin.|UNDISCLOSED|secondary|   2343|    yes|  no|    1042|    yes|
| 56|     admin.|    married|secondary|   1074|     no|  no|    1467|    yes|
| 41| technician|    married|secondary|   1270|    yes|  no|    1389|    yes|
| 55|   services|UNDISCLOSED|secondary|   1074|    yes|  no|     579|    yes|
| 54|     admin.|    married| tertiary|    184|     no|  no|     673|    yes|
| 42| management|UNDISCLOSED| tertiary|      0|    yes| yes|     562|    yes|
| 56| management|    married| tertiary|    830|    yes| yes|    1201|    yes|
| 60|    retired|UNDISCLOSED|secondary|    545|    yes|  no|    1030|    yes|
| 37| technician|    married|secondary|      1|    yes|  no|     608|    yes|
| 28|   services|UNDISCLOSED|secondary|   1074|    yes|  no|    

Great work! Our DataFrame now has zero missing values. Try figuring out a way to prove that there are no missing values.

At this stage, we're done with DataFrames. Please remember that this material will take some time to sink in, so keep practicing and reading documentation! 